In [1]:
import sys
sys.path.insert(0, '/home/cameron/Projects/hypso-package')

## Read NetCDF File

In [2]:
# Import Satellite Object
from hypso import Hypso1
import os

# Define HYPSO Image File and the .points from QGIS (if available)
dir_path = '/home/cameron/Dokumenter/Data/frohavet'
nc_file = os.path.join(dir_path, 'frohavet_2024-05-06_1017Z-l1a.nc')
points_file = os.path.join(dir_path, 'frohavet_2024-05-06_1017Z-bin3.points')

# Create Satellite Object
satobj = Hypso1(hypso_path=nc_file, points_path=points_file, verbose=True)

[INFO] Capture spatial dimensions: (598, 1092)
[INFO] Capture capture type: wide
[INFO] Running georeferencing...
No image mode provided. Detected image mode: bin3
[INFO] Using UTM map: WGS 84 / UTM zone 32N EPSG: 32632


## Generate L1B datacube

In [3]:
satobj.generate_l1b_cube()

[INFO] Running calibration routines...
[INFO] Running radiometric calibration...
[INFO] Running smile correction...
[INFO] Running destriping correction...


## Generate chlorophyll estimates

In [4]:
satobj.generate_chlorophyll_estimates('band_ratio')

[INFO] Running band ratio chlorophyll estimation...
[INFO] Calibration has already been run. Skipping.


## Load area definition

In [5]:
from pyresample import load_area
area_def = load_area("./frohavet_demo.yaml")

## Get L1A SatPy scene

In [6]:
l1a_scene = satobj.get_l1a_satpy_scene()

## Get chlorophyll estimates SatPy scene

In [7]:
chl_scene = satobj.get_chlorophyll_estimates_satpy_scene()

## Resample capture to area defintion

In [8]:
import numpy as np

resampled_l1a_scene = l1a_scene.resample(area_def, resampler='bilinear', fill_value=np.NaN)
resampled_chl_scene = chl_scene.resample(area_def, resampler='bilinear', fill_value=np.NaN)

/home/cameron/miniforge3/lib/python3.10/site-packages/pyproj/crs/crs.py:1286: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


In [18]:
ds = resampled_l1a_scene[['band_100', 'band_90']].to_xarray()
da = ds.to_stacked_array('band', sample_dims=['lines','samples'])
da.to_numpy().shape

TypeError: unhashable type: 'list'

In [9]:
resampled_l1a_scene['band_100']

<xarray.DataArray 'reshape-298b18adb41984022fa85ccffe39cd31' (y: 1173, x: 611)>
dask.array<reshape, shape=(1173, 611), dtype=float64, chunksize=(1173, 611), chunktype=numpy.ndarray>
Coordinates:
  * x        (x) float64 -9.737e+04 -9.707e+04 ... 8.556e+04 8.586e+04
  * y        (y) float64 -2.631e+06 -2.631e+06 ... -2.983e+06 -2.983e+06
    crs      object PROJCRS["unknown",BASEGEOGCRS["unknown",DATUM["Unknown ba...
Attributes: (12/16)
    level:                L1a
    units:                a.u.
    description:          Raw sensor value
    name:                 band_100
    _satpy_id:            DataID(name='band_100')
    file_type:            None
    ...                   ...
    end_time:             2024-05-06 10:17:00
    modifiers:            ()
    ancillary_variables:  []
    wavelength:           99 band (99-99 band)
    band:                 99
    area:                 Area ID: frohavet\nDescription: roi\nProjection: {'...

In [13]:
l1a_scene['band_36']

<xarray.DataArray (y: 598, x: 1092)>
array([[ 9025,  9168,  9252, ...,  8174,  8122,  8186],
       [ 8993,  9180,  9128, ...,  8175,  8081,  8170],
       [ 8983,  9125,  9132, ...,  8108,  8068,  8105],
       ...,
       [11937, 12207, 12134, ..., 36855, 36855, 36855],
       [12005, 12210, 12212, ..., 36855, 36855, 36855],
       [12014, 12260, 12218, ..., 36855, 36855, 36855]], dtype=uint16)
Dimensions without coordinates: y, x
Attributes: (12/16)
    level:                L1a
    units:                a.u.
    description:          Raw sensor value
    name:                 band_36
    _satpy_id:            DataID(name='band_36')
    file_type:            None
    ...                   ...
    end_time:             2024-05-06 10:17:00
    modifiers:            ()
    ancillary_variables:  []
    wavelength:           35 band (35-35 band)
    band:                 35
    area:                 Shape: (598, 1092)\nLons: <xarray.DataArray (y: 598...

In [12]:
resampled_l1a_scene['latitude']

KeyError: "No dataset matching 'DataQuery(name='latitude')' found"

In [20]:
resampled_l1a_scene.unload('latitude')
resampled_l1a_scene.unload('longitude')

In [2]:
resampled_l1a_scene.__delitem__('latitude')

NameError: name 'resampled_l1a_scene' is not defined

In [1]:
from satpy import Scene